# Archeo

## Training modelli

### Caricamento di path e librerie, oltre allo scheletro della configurazione del modello

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning
!pip install segmentation-models-pytorch
!pip install rasterio
!pip install gdal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=0a62c6db41ee628aab608490ec4161333af60771287dfc7fefc2f503faa8c47c
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=ff4aeebc4a47a9968ae9b2db1e11c79eb1745da743f71b74568ad00a8e90e82d
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8

In [ ]:
import os
import random
import numpy as np
import pandas as pd
import rich

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations import pytorch

import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import TQDMProgressBar, RichProgressBar

import segmentation_models_pytorch  as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

from datetime import datetime

from torchvision.transforms import functional as F
from torchvision.transforms import AutoAugment, AutoAugmentPolicy

from matplotlib import pyplot as plt
import torchvision.transforms as T
import torchvision.transforms.autoaugment as v2
import numpy as np
import os
import cv2
from PIL import Image
import albumentations as A

In [ ]:
local = False

LOCAL_URL = ""
CLOUD_URL = "/content/drive/My Drive/DatiModello/"

base_url = ""

if(local):
  base_url = LOCAL_URL
else:
  base_url = CLOUD_URL

In [ ]:
PATH_LOG = base_url + 'exp_logs/'
PATH_DATASETS = base_url + 'datasets/'
PATH_OUTPUT = base_url + 'outputs/'

modelli = ['abu_1k', 'abu_2k']
testset_modelli = {}

config = {
    "timestamp" : datetime.now().strftime("%d-%m-%Y_%H%M%S"),
    "dataset_path" : "",
    "checkpoint_path" : PATH_LOG,
    "random_seed" : 1234,
    "arch" : "MAnet", #Unet,MAnet
    "encoder" : "efficientnet-b3", #resnet18, dpn68, efficientnet-b3
    "weights" : "imagenet",
    "loss" : "focal",
    "learning_rate" : 0.0001,
    "precision" : 32,
    "epochs" : 20,
    "batch_size" : 8,
    "dim_input" : '',
    "in_channels" : 3
}

random.seed(config["random_seed"])
np.random.seed(config["random_seed"])
torch.manual_seed(config["random_seed"])
print(PATH_DATASETS)
print(PATH_LOG)

/content/drive/My Drive/DatiModello/datasets/
/content/drive/My Drive/DatiModello/exp_logs/


### Divisione del dataset in train set (80%), validation set (10%) e test set (10%)

In [ ]:
def load_dataset(dataset_path, SEED, indices):
    images_directory = dataset_path + "sites/"
    masks_directory = dataset_path + "masks/"

    filenames_train = np.asarray(list(sorted(os.listdir(images_directory))))
    print("total files:", len(filenames_train))

    valid_split = -int(len(indices)*0.2)
    test_split = valid_split//2

    train_indices = indices[:valid_split]
    valid_indices = indices[valid_split:test_split]
    test_indices = indices[test_split:]
    train_images_filenames = filenames_train[train_indices]
    val_images_filenames = filenames_train[valid_indices]
    test_images_filenames = filenames_train[test_indices]

    print(
        "root:", base_url,
        "\n images", images_directory,
        "\n masks", masks_directory,"\n ---",
        '\n train images', len(train_images_filenames),
        '\n val images', len(val_images_filenames),
        '\n test images', len(test_images_filenames),
        '\n ---\ntotal images', len(filenames_train)
    )

    print("empty masks percentage: %.4f %.4f %.4f" %
          ( np.sum([i.startswith("neg") for i in train_images_filenames]) / len(train_images_filenames),
            np.sum([i.startswith("neg") for i in val_images_filenames]) /   len(val_images_filenames),
            np.sum([i.startswith("neg") for i in test_images_filenames]) /  len(test_images_filenames)
          ))

    return (
        images_directory,
        masks_directory,
        train_images_filenames,
        val_images_filenames,
        test_images_filenames
    )

In [ ]:
# Crop e resize dipendono dalla dimensione dell'input
EqualizeString = "TECNICA EQUALIZE : mode='cv', by_channels=True, mask=None, mask_params=(), always_apply=False, p=0.5"
ZoomBlurString = "TECNICA ZOOMBLUR : max_factor=1.5, step_factor=(0.01, 0.03), always_apply=False, p=0.7"
SharpenString = "TECNICA SHARPEN : alpha=(0.2, 0.5),lightness=(0.5, 1.0),always_apply=False,p=0.5"

templog = [EqualizeString, ZoomBlurString, SharpenString]

def esegui_trasformazioni(indiceTrasformazione):
    if config["dim_input"] == '1k':
        if indiceTrasformazione == 0:
            train_transform = A.Compose(
                [
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(512, 512, p=1.0),
                    A.Resize(256, 256),
                    A.pytorch.ToTensorV2()
                ]
            )
        elif indiceTrasformazione == 1:
            train_transform = A.Compose(
                [
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )

            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(512, 512, p=1.0),
                    A.Resize(256, 256),
                    A.pytorch.ToTensorV2()
                ]
            )
        else:
            train_transform = A.Compose(
                [
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(512, 512, p=1.0),
                    A.Resize(256, 256),
                    A.pytorch.ToTensorV2()
                ]
            )
    elif config["dim_input"] == '2k':
        if indiceTrasformazione == 0:
            train_transform = A.Compose(
                [
                    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.25),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(1024, 1024, p=1.0),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
        elif indiceTrasformazione == 1:
            train_transform = A.Compose(
                [
                    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.25),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(1024, 1024, p=1.0),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
        else:
            train_transform = A.Compose(
                [
                    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), always_apply=True),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )
            # le immagini di validazione non sono affette da tecniche di data augmentation
            val_transform = A.Compose(
                [
                    A.RandomCrop(1024, 1024, p=1.0),
                    A.Resize(512, 512),
                    A.pytorch.ToTensorV2()
                ]
            )

    return train_transform, val_transform


### Caricamento del dataset con le relative trasformazioni

In [ ]:
# Carico il dataset con le relative trasformazioni, in base al tipo di modello che stiamo analizzando,
# cioè in base alla dimensione di input di ogni immagine e ai canali di input.

class ArcheoDataset(Dataset):
    def __init__(self, images_filenames, images_directory, masks_directory, transform=None):
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.masks_directory = masks_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image_path = self.images_directory + image_filename
        mask_path = self.masks_directory +  image_filename.replace(".jpg", ".png")

        image = np.array(Image.open(image_path))#.convert("RGB"))

        mask = ~np.array(Image.open(mask_path).convert("L")) # masks are flipped because of qgis
        mask = mask.astype("float")
        mask[mask > 0.0] = 1.0
        mask = np.expand_dims(mask, -1)

        if(self.transform is not None):
          transformed = self.transform(image=image, mask=mask)
          image = transformed["image"]
          mask = transformed["mask"].permute(2,0,1)
        else: # se non viene impostata una trasformazione va comunque effettuato il resize
          trasformazione = A.Compose([A.Resize(512,512), A.pytorch.ToTensorV2()])
          transformed = trasformazione(image=np.asarray(image), mask=np.asarray(mask))
          image = transformed["image"]
          mask = transformed["mask"].permute(2,0,1)

        return image, mask, image_filename

### Inizializzazione del modello


In [ ]:
# Inizializzazione del modello, calcolo della funzione di loss sulla maschere, probabilità ottenute
# con il sigmoid. Le probabilità maggiori di 0.5 vengono trasformate in 1, quelle sotto lo 0.5 in 0.

class ArcheoModel(pl.LightningModule):

    def __init__(self, arch, encoder_name, in_channels, out_classes, **kwargs):
        super().__init__()
        #self.model = smp.create_model(
        #    arch, encoder_name=encoder_name, in_channels=in_channels, classes=out_classes, **kwargs
        #)
        params = smp.encoders.get_preprocessing_params(encoder_name)

        # SECONDO APPROCCIO
        self.model = smp.MAnet(
          encoder_name = encoder_name,
          encoder_weights = "imagenet",
          in_channels = in_channels,
          classes = out_classes,
        )
        # self.preprocess_input = get_preprocessing_fn(encoder_name, pretrained='imagenet')

        self.register_buffer("std", torch.tensor(params["std"]).view(1, 3, 1, 1)) # 3 sta per RGB se grayscale metti 1 oppure utilizza in_channels
        self.register_buffer("mean", torch.tensor(params["mean"]).view(1, 3, 1, 1)) # stesso per std
        self.validation_step_outputs = []
        self.training_step_outputs = []
        self.test_step_outputs = []


        if config["loss"] == "jaccard":
            self.loss_fn = smp.losses.JaccardLoss(smp.losses.BINARY_MODE, from_logits=True)
        if config["loss"] == "dice":
            self.loss_fn = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits=True)
        if config["loss"] == "focal":
            self.loss_fn = smp.losses.FocalLoss(mode=smp.losses.BINARY_MODE)


    def forward(self, image):
        image = (image - self.mean) / self.std
        mask = self.model(image)
        return mask

    def shared_step(self, batch, stage):
        image = batch[0] # totry batch["image"]

        # (batch_size, num_channels, height, width)
        assert image.ndim == 4

        # Check that image dimensions are divisible by 32
        h, w = image.shape[2:]
        assert h % 32 == 0 and w % 32 == 0

        mask = batch[1] # totry batch["mask"]

        assert mask.ndim == 4 # [batch_size, num_classes, height, width] = dim 4

        # Check that mask values in between 0 and 1, NOT 0 and 255 for binary segmentation
        assert mask.max() <= 1.0 and mask.min() >= 0

        logits_mask = self.forward(image) # -> output grezzo del modello
        loss = self.loss_fn(logits_mask, mask) # calcolo funzione di loss = errore
        self.log_dict({f"{stage}/loss": loss.detach().item()}, batch_size=config["batch_size"]) # log metrica loss
        prob_mask = logits_mask.sigmoid() # funzione di attivazione -> probabilità pixel per pixel
        pred_mask = (prob_mask > 0.5).float()
        pred_mask = pred_mask.permute(0, 3, 1, 2) #  "NCHW" (dove N è la dimensione del batch, C è il numero di canali, H è l'altezza e W è la larghezza)
        mask = mask.permute(0, 3, 1, 2)

        tp, fp, fn, tn = smp.metrics.get_stats(pred_mask.long(), mask.long(), mode="binary")

        if stage == "train":
            self.log_dict(
                {
                  "train/batch-IOU-img" : smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro-imagewise"),
                  "train/batch-IOU" : smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")
                },
                prog_bar=True,
                batch_size=config["batch_size"]
            )

        return {
            "loss": loss,
            "tp": tp,
            "fp": fp,
            "fn": fn,
            "tn": tn,
        }

    def shared_epoch_end(self, stage):
        if(stage == "train"):
          tp = torch.cat([x["tp"] for x in self.training_step_outputs])
          fp = torch.cat([x["fp"] for x in self.training_step_outputs])
          fn = torch.cat([x["fn"] for x in self.training_step_outputs])
          tn = torch.cat([x["tn"] for x in self.training_step_outputs])
        elif(stage == "valid"):
          tp = torch.cat([x["tp"] for x in self.validation_step_outputs])
          fp = torch.cat([x["fp"] for x in self.validation_step_outputs])
          fn = torch.cat([x["fn"] for x in self.validation_step_outputs])
          tn = torch.cat([x["tn"] for x in self.validation_step_outputs])
        else:
          tp = torch.cat([x["tp"] for x in self.test_step_outputs])
          fp = torch.cat([x["fp"] for x in self.test_step_outputs])
          fn = torch.cat([x["fn"] for x in self.test_step_outputs])
          tn = torch.cat([x["tn"] for x in self.test_step_outputs])

        per_image_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro-imagewise")
        dataset_iou = smp.metrics.iou_score(tp, fp, fn, tn, reduction="macro")

        metrics = {
            f"{stage}/IOU-img": per_image_iou,
            f"{stage}/IOU": dataset_iou,
        }

        self.log_dict(metrics, prog_bar=True, batch_size=config["batch_size"])


    # BASIC STEP
    def training_step(self, batch, batch_idx):
        output = self.shared_step(batch, "train")
        self.training_step_outputs.append(output)
        return output

    def validation_step(self, batch, batch_idx):
        output = self.shared_step(batch, "valid")
        self.validation_step_outputs.append(output)
        return output

    def test_step(self, batch, batch_idx):
        output = self.shared_step(batch, "test")
        self.test_step_outputs.append(output)
        return output


    # EPOCH END
    def on_training_epoch_end(self):
        self.shared_epoch_end("train")
        return self.training_step_outputs.clear()

    def on_validation_epoch_end(self):
        self.shared_epoch_end("valid")
        return self.validation_step_outputs.clear()

    def on_test_epoch_end(self):
        self.shared_epoch_end("test")
        return self.test_step_outputs.clear()

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=config["learning_rate"]) # 0.00005


In [ ]:
def set_config(name_modello):
  config["dataset_path"] = PATH_DATASETS + name_modello + "/"
  if '1k' in name_modello:
    config["dim_input"]='1k'
  else:
    config["dim_input"]='2k'

### Training di ogni modello in base alle sue caratteristiche

In [ ]:
# 10 test, con diverse trasformazioni, per ogni modello.
def chiamataTest(k):
    iou_test_modelli = {}
    for i in range(len(modelli)):
        set_config(modelli[i])
        filenames_train = np.asarray(list(sorted(os.listdir(os.path.join(config["dataset_path"], "sites/")))))
        print("total files:", len(filenames_train))
        indices = np.arange(0, len(filenames_train))
        np.random.shuffle(indices)
        print(indices)

        name_ckpt = os.listdir(PATH_LOG + 'lightning_logs/' + modelli[i] + '/checkpoints/')[0]
        model = ArcheoModel.load_from_checkpoint(
            arch=config["arch"],
            encoder_name=config["encoder"],
            encoder_weights=config["weights"],
            in_channels=config['in_channels'],
            out_classes=1,
            checkpoint_path=PATH_LOG + 'lightning_logs/' + modelli[i] + '/checkpoints/' + name_ckpt
        )

        print(config['dataset_path'])
        images_directory, masks_directory, train_images_filenames, val_images_filenames, test_images_filenames = load_dataset(
            config["dataset_path"], config["random_seed"], indices
        )

        testiou = []
        for j in range(10):
            print("Risultato " + str(j) + " del modello " + modelli[i])
            train_transform, val_transform = esegui_trasformazioni(k)
            test_dataset = ArcheoDataset(test_images_filenames, images_directory, masks_directory, transform=val_transform)
            test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=False, num_workers=1)
            trainer = pl.Trainer(
                max_epochs=40,
                precision='16-mixed',
                accelerator="auto"
            )
            test_metrics = trainer.test(model, dataloaders=test_loader, verbose=True)
            testiou.append(test_metrics[0]['test/IOU-img'])

        iou_test_modelli[modelli[i]] = testiou

    # Stampo le statistiche per ogni modello (mean, min, max, std)
    for i in modelli:
        x = np.array(iou_test_modelli[i])
        print("\n Le statistiche sul test set per il modello " + i + " sono:")
        print("media: " + str(round(x.mean(), 4)) + " | min: " + str(round(x.min(), 4)) + " | max: " + str(
            round(x.max(), 4)) + " | std: " + str(round(x.std(), 4)))

    log_path = '/content/drive/My Drive/LOGTEST/log.txt'

    with open(log_path, 'a') as log_file:
        log_file.write("\n " + templog[k])
        for i in modelli:
            x = np.array(iou_test_modelli[i])
            log_file.write("\n Le statistiche sul test set per il modello " + i + " sono:")
            log_file.write("media: " + str(round(x.mean(), 4)) + " | min: " + str(round(x.min(), 4)) +
                           " | max: " + str(round(x.max(), 4)) + " | std: " + str(round(x.std(), 4)))


In [ ]:
for k in range(3):
    print("Tecnica numero "+ str(k))
    for j in range(10):
        print("RUN NUMERO "+ str(j)+ " DI "+ templog[k])
        # Ogni modello viene addestrato in base alle sue caratteristiche.
        for i in range(len(modelli)):
            set_config(modelli[i])
            filenames_train = np.asarray(list(sorted(os.listdir(os.path.join(config["dataset_path"], "sites/")))))
            print("total files:", len(filenames_train))
            indices = np.arange(0, len(filenames_train))
            np.random.shuffle(indices)
            print(indices)

            images_directory, masks_directory, train_images_filenames, val_images_filenames, test_images_filenames = load_dataset(config["dataset_path"], config["random_seed"], indices)
            train_transform, val_transform = esegui_trasformazioni(k)

            train_dataset = ArcheoDataset(train_images_filenames, images_directory, masks_directory, transform=train_transform)
            val_dataset = ArcheoDataset(val_images_filenames, images_directory, masks_directory, transform=val_transform)

            train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True, num_workers=1)
            val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=False, num_workers=1)

            # creazione del modello
            model = ArcheoModel(config["arch"], encoder_name=config["encoder"], encoder_weights=config["weights"], in_channels=config['in_channels'], out_classes=1)

            # training del modello
            trainer = pl.Trainer (
                max_epochs=config["epochs"],
                precision=config["precision"],
                accelerator="auto",
                logger=pl_loggers.TensorBoardLogger(config["checkpoint_path"]),
                log_every_n_steps=1,
                enable_progress_bar=True,
                callbacks=[RichProgressBar(refresh_rate=1)]
            )

            cfg_text = "\n".join([str(key) + " : **" + str(config[key]) + "**  " for key in config])
            print(cfg_text)
            trainer.logger.experiment.add_text(tag="config", text_string=cfg_text)

            trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
            os.rename(PATH_LOG + 'lightning_logs/version_0', PATH_LOG + 'lightning_logs/' + modelli[i])
        chiamataTest(k)


Epoch 3/19 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 0:00:29 • 0:00:00 0.86it/s v_num: 0.000 train/batch-IOU-img:   
                                                                               0.595 train/batch-IOU: 0.036        
                                                                               valid/IOU-img: 0.114 valid/IOU:     
                                                                               0.125                               
Validation ━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 2/4   0:00:01 • 0:00:02 1.11it/s

## Test di ogni modello


In [ ]:
from pytorch_lightning.loggers import tensorboard
%reload_ext tensorboard
%tensorboard --logdir=PATH_LOG

## Funzioni ausiliare al main

### Caricamento dei csv contenenti coordinate siti


In [ ]:
import csv
import seaborn as sns
import json


sns.set_style("whitegrid")
namesite2Centroid={}
nameneg2Centroid={}
namemaysan2Centroid={}
name2min={}


with open(base_url + 'dataset.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
      namesite2Centroid[row[0]]=[row[2],row[3]]


with open(base_url + 'negs110.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if "entry" in row[0]:
          nameneg2Centroid[row[0]]=[row[1],row[2]]
          continue

        solo_numeri = ''.join(c for c in row[0] if c.isdigit())
        entry = float(solo_numeri[:-1])

        if entry > 109:
          entry = row[0][:-2]
          nameneg2Centroid[entry]=[row[1],row[2]]
        else:
          nameneg2Centroid[row[0]]=[row[1],row[2]]

### Generazione, modifica e salvataggio predizioni

In [ ]:
from scipy.ndimage import gaussian_filter
import cv2

# data la predizione,viene applicato un cutoff
# ibatch rappresenta l'i-esimo batch composto da 3 elementi:
#  - ibatch[0] -> immagini in input
#  - ibatch[1] -> sono le maschere di ground truth associate
#  - ibatch[2] -> sono le previsioni del modello
#  - ipr_masks_11 -> sono le maschere di probabilità (logits.sigmoid())

def stampa_predizioni_cutoff(ibatch, ipr_masks_11, nome_modello, cutoff_val):
    for i, (input_image, groundtruth_mask, prob_mask, image_filename) in enumerate(zip(ibatch[0], ibatch[1], ipr_masks_11, ibatch[2])):
      prob_mask = prob_mask.numpy().squeeze()

      # cutoff
      image_cutoff = prob_mask.copy()
      image_cutoff = gaussian_filter(image_cutoff, sigma=5)
      image_cutoff = ((image_cutoff + 0.5)**2) - 0.5
      image_cutoff[image_cutoff <= cutoff_val] = 0.0
      image_cutoff[image_cutoff > cutoff_val] = 1.0

      output_directory = PATH_OUTPUT + nome_modello + '/pred_siti_tronc'+ str(cutoff_val) + '/'
      output_filename = image_filename[:-4] + '.png'
      output_path = os.path.join(output_directory, output_filename)

      os.makedirs(output_directory, exist_ok=True)

      plt.imsave(output_path, image_cutoff, cmap="magma", vmin=0.0, vmax=1.0)
      plt.show()

### Dall'immagine al tif -> CreateTif + convertImageToTif

In [ ]:
# Dato la predizione, viene creato il tif associato.
import rasterio

def createTif(ovest,sud,est,nord,patin,patout):
  dataset = rasterio.open(patin, 'r')
  bands = [1, 2, 3]
  data = dataset.read(bands)
  transform = rasterio.transform.from_bounds(ovest, sud, est, nord, data.shape[2], data.shape[1])
  crs = {'init': 'epsg:3857'}

  with rasterio.open(patout, 'w', driver='GTiff', width=data.shape[2], height=data.shape[1], count=3, dtype=data.dtype, nodata=0, transform=transform, crs=crs) as dst:
      dst.write(data, indexes=bands)

In [ ]:
# Ricostruisco la predizione usando le trasformazioni salvate in precedenza.
def convertImageToTif(nome_modello, cutoff_val):
  for i in range(len(test_images_filenames)):

    path_predizioni = PATH_OUTPUT + nome_modello + '/pred_siti_tronc'+ str(cutoff_val) + '/'
    output_directory = PATH_OUTPUT + nome_modello + '/tif' + str(cutoff_val) + '/'

    os.makedirs(output_directory, exist_ok=True)

    im_sito = Image.open(path_predizioni + test_images_filenames[i].replace(".jpg", ".png"))

    newsize = (1024, 1024)
    im_sito = im_sito.resize(newsize)

    if("neg" in test_images_filenames[i]):
      x_centroide=float(nameneg2Centroid[test_images_filenames[i][:-4]][0])
      y_centroide=float(nameneg2Centroid[test_images_filenames[i][:-4]][1])
    else:
      x_centroide=float(namesite2Centroid[test_images_filenames[i][:-4]][0])
      y_centroide=float(namesite2Centroid[test_images_filenames[i][:-4]][1])

    ovest=x_centroide-1000
    est=x_centroide+1000
    sud=y_centroide-1000
    nord=y_centroide+1000
    createTif(ovest=ovest, sud=sud, est=est, nord=nord,
              patin = path_predizioni + test_images_filenames[i].replace(".jpg", ".png"),
              patout = output_directory + test_images_filenames[i].replace(".jpg", ".tif"))




### Dai tif agli shapefile

In [ ]:
# prende i tif files e restituisce gli shape files
import os
import subprocess
from tqdm.auto import tqdm
from osgeo import gdal, ogr, osr
# from gdal import gdal_contour


def convertTifToShape(tif_path, shape_path):

  # mapping between gdal type and ogr field type
  type_mapping = { gdal.GDT_Byte: ogr.OFTInteger,
                   gdal.GDT_UInt16: ogr.OFTInteger,
                   gdal.GDT_Int16: ogr.OFTInteger,
                   gdal.GDT_UInt32: ogr.OFTInteger,
                   gdal.GDT_Int32: ogr.OFTInteger,
                   gdal.GDT_Float32: ogr.OFTReal,
                   gdal.GDT_Float64: ogr.OFTReal,
                   gdal.GDT_CInt16: ogr.OFTInteger,
                   gdal.GDT_CInt32: ogr.OFTInteger,
                   gdal.GDT_CFloat32: ogr.OFTReal,
                   gdal.GDT_CFloat64: ogr.OFTReal }

  filenames = os.listdir(tif_path)



  os.makedirs(shape_path, exist_ok=True)
  print("Totale tif: " + str(len(filenames)))
  for f in tqdm(filenames):
    print(f[:-4])
    # subprocess.run(["gdal_contour.exe", "-i", "128", "-p", tif_path + f, shape_path + f[:-4] + ".shp"], capture_output=True, shell=True, check=False)

    # Apre il dataset raster
    src_ds = gdal.Open(tif_path + f)
    src_band = src_ds.GetRasterBand(1)  # default band 1

    # Crea un vettore per i poligoni di contorno
    dst_layer = None

    # Creazione del driver per lo shapefile
    driver = ogr.GetDriverByName("ESRI Shapefile")

    # Crea il dataset shapefile
    dst_ds = driver.CreateDataSource(shape_path + f[:-4] + ".shp")

    # Crea il layer
    spatial_ref = osr.SpatialReference()
    spatial_ref.ImportFromEPSG(3857)
    dst_layer = dst_ds.CreateLayer(f[:-4], srs=spatial_ref, geom_type=ogr.wkbPolygon)

    raster_field = ogr.FieldDefn('elevation', type_mapping[src_band.DataType]) ## aggiunto
    dst_layer.CreateField(raster_field)


    # Copia i poligoni nel layer
    # gdal.Polygonize(src_band, None, dst_layer, -1, [], callback=None) # eliminato
    gdal.Polygonize(src_band, None, dst_layer, 0, [], callback=None) ## aggiunto

    # Chiude il dataset raster e il dataset shapefile
    src_ds = None
    dst_ds = None


### Visualizzazione immagine, groundtruth mask e maschera predetta

In [ ]:
def view_image_gtMask_pred(image, gt_mask, image_cutoff):
  plt.figure(figsize=(10, 5))

  plt.subplot(1, 3, 1)
  plt.imshow(image.numpy().transpose(1, 2, 0))  # convert CHW -> HWC
  plt.title("Image")
  plt.axis("off")

  plt.subplot(1, 3, 2)
  plt.imshow(gt_mask.numpy().squeeze()) # just squeeze classes dim, because we have only one class
  plt.title("Ground truth")
  plt.axis("off")

  plt.subplot(1, 3, 3)
  plt.imshow(image_cutoff) # just squeeze classes dim, because we have only one class
  plt.title("Prediction")
  plt.axis("off")

  plt.show()

### Da shapefile a geojson (utilizzato poi per calcolare la matrice di confusione) -> testForIntersection e convertShapeToGeojson

In [ ]:
import geopandas as geopd
from shapely.geometry.multipolygon import MultiPolygon

# Assegna ad ogni sito un valore tra (tp, tn, fp, fn) in base all'intersezione della forma predetta con la forma originale
def testForIntersection(site_id, path, verbose=False):
    if verbose: print("Processing site: " + site_id)

    ### get shape for the original site
    sites = geopd.read_file(base_url + "shapefile/RS ABU-GHRAIB.shp").to_crs("EPSG:3857")
    a = sites[sites.entry_id == site_id][["entry_id", "geometry"]]

    if verbose: print('Loading Contours')

    b = geopd.read_file(path + site_id + ".shp").set_crs("EPSG:3857")
    b.geometry = b.geometry.convex_hull
    b["entry_id"] = "pred"
    ### alla creazione dello shapefile, viene assegnato un valore di elevation per contraddistinguere le parti dello shape che sono regioni (poligoni)
    ### validi da quelli che rappresentano lo "sfondo" tuttavia non capisco perchè per le shape0.2 quelle con valore 0 siano i poligoni regolari mentre
    ### per le shape0.5 sia il contrario
    if("0.2" in path):
      b = b[b['elevation'] == 0]
    elif("0.5" in path):
      b = b[b['elevation'] != 0]

    if verbose:
        print("Number of features:", len(b))
        print(b)

    if len(b) > 1: # if there is more than one shape
        b["geometry"] = MultiPolygon([feature for feature in b["geometry"]])
        b = b[:1].copy()
        if verbose:
            print(b)
            b.plot()
    # negs should have no geometry
    if len(b) == 0 and site_id.startswith("neg"):
        if verbose: print("good neg")
        return "TN", site_id, None

    # if neg has geometry then FP
    elif len(b) > 0 and site_id.startswith("neg"):
        if verbose: print("false positive")
        return "FP", site_id, b.iloc[0]["geometry"] #False

    # if no geometry and not neg then FN. use a.geometry for use in QGIS
    elif len(b) == 0 and not site_id.startswith("neg"):
        if verbose: print("false negative")
        return "FN", site_id, a.iloc[0]["geometry"]

    # compute intersection
    if ~b.iloc[0].geometry.is_valid:
        b.geometry = b.geometry.buffer(0)
    intersects = a.iloc[0]["geometry"].intersects(b.iloc[0]["geometry"])
    if verbose:
        c = pd.concat([a,b])
        c.plot(column="entry_id", legend=True, figsize=(5,5), cmap="Set3")
        print("INTERSECTION: ", intersects)

    if intersects: # right geometry
        return "TP", site_id, b.iloc[0]["geometry"]
    else: # wrong geometry
        return "FP", site_id, b.iloc[0]["geometry"]

In [ ]:
# prende in input gli shapefile dei siti e ritorna un geojson
# contenente id, nome sito, geometria e valore tra tp,tn,fp,fn

def convertShapeToGeojson(testset, path_in, path_out):
 res = {"index":[], "entry_id":[], "geometry":[], "cat":[]}
 indice = 0
 for sito in testset:
    sito = sito[:-4]
    cat, eid, geom = testForIntersection(sito, path_in, verbose = True)
    res["index"].append(indice)
    res["entry_id"].append(eid)
    res["geometry"].append(geom)
    res["cat"].append(cat)
    indice += 1
 res_df = geopd.GeoDataFrame(res)
 res_df = res_df.set_index("index")
 res_df.to_file(path_out, driver='GeoJSON', crs="EPSG:3857")

# MAIN


In [ ]:
modelli_da_comparare=['abu_1k', 'abu_2k']


for modello in modelli_da_comparare:
  set_config(modello)

  ### load datasets, it has to be used when executing only this cell in order to retriev dataset info ###
  # filenames_train = np.asarray(list(sorted(os.listdir(os.path.join(config["dataset_path"], "sites/")))))
  # print("total files:", len(filenames_train))
  # indices = np.arange(0, len(filenames_train))
  # np.random.shuffle(indices)
  # print("test indices: ", indices)
  # images_directory, masks_directory, train_images_filenames, val_images_filenames, test_images_filenames = load_dataset(config["dataset_path"], config["random_seed"], indices)
  ### end ###

  name_ckpt = os.listdir(PATH_LOG + 'lightning_logs/' + modello + '/checkpoints/')[0]

  model= ArcheoModel.load_from_checkpoint(
      arch = config["arch"],
      encoder_name = config["encoder"],
      encoder_weights = config["weights"],
      in_channels = config['in_channels'],
      out_classes = 1,
      checkpoint_path = PATH_LOG + 'lightning_logs/' + modello + '/checkpoints/' + name_ckpt
  )

  random.seed(config["random_seed"])
  np.random.seed(config["random_seed"])
  torch.manual_seed(config["random_seed"])

  print(test_images_filenames)

  train_transform, val_transform = esegui_trasformazioni()

  test_dataset = ArcheoDataset(
      test_images_filenames,
      images_directory,
      masks_directory
  )

  test_dataloader = DataLoader(test_dataset, batch_size = config["batch_size"], shuffle=False, drop_last=False, num_workers=1)

  it = iter(test_dataloader)

  for j in range(len(it)):
    batch = next(it)
    with torch.no_grad():
      model.eval()
      logits = model(batch[0].cuda())
    pr_masks = logits.sigmoid().detach().cpu()
    stampa_predizioni_cutoff(batch, pr_masks, modello, 0.2)
    stampa_predizioni_cutoff(batch, pr_masks, modello, 0.5)


  batch = next(iter(test_dataloader))
  with torch.no_grad():
      model.eval()
      logits = model(batch[0].cuda())
  pr_masks = logits.sigmoid().detach().cpu()

  for image, gt_mask, pr_mask in zip(batch[0], batch[1], pr_masks):
      prob_mask = pr_mask.numpy().squeeze()

      # cutoff
      image_cutoff = prob_mask.copy()
      image_cutoff = gaussian_filter(image_cutoff, sigma=5)
      image_cutoff = ((image_cutoff + 0.5)**2) - 0.5
      image_cutoff[image_cutoff <= 0.5] = 0.0
      image_cutoff[image_cutoff > 0.5] = 1.0

      view_image_gtMask_pred(image, gt_mask, image_cutoff)


  convertImageToTif(modello, 0.2)
  convertImageToTif(modello, 0.5)
  convertTifToShape(PATH_OUTPUT + modello + '/tif0.2/', PATH_OUTPUT + modello + '/shape0.2/')
  convertTifToShape(PATH_OUTPUT + modello + '/tif0.5/', PATH_OUTPUT + modello + '/shape0.5/')

  os.makedirs(PATH_OUTPUT + modello + '/GeoJson', exist_ok=True)
  convertShapeToGeojson(test_images_filenames, PATH_OUTPUT + modello + '/shape0.2/', PATH_OUTPUT + modello + '/GeoJson/preds02.geojson')
  convertShapeToGeojson(test_images_filenames, PATH_OUTPUT + modello + '/shape0.5/', PATH_OUTPUT + modello + '/GeoJson/preds05.geojson')


## Analisi risultati

### Matrice di confusione (adjusted deriva dalla consultazione con archeologia)

In [ ]:
def calculate_matrix(preds):
  tp = preds[preds.cat == "TP"]['entry_id'].shape[0]
  tn = preds[preds.cat == "TN"]['entry_id'].shape[0]
  fp = preds[preds.cat == "FP"]['entry_id'].shape[0]
  fn = preds[preds.cat == "FN"]['entry_id'].shape[0]

  matrix = [tp, tn, fp, fn]


  # adjusted

  # sono siti non visibili, quindi classificati erroneamente come tn
  # fn2tn = preds[(preds.cat == "FN")&(preds.correction == "TN")]['entry_id'].shape[0]

  # sites_inside_ot = preds[(preds.notes.str.contains('INSIDE OTHER', na=False)) ].shape[0]

  # siti non visibili e il modello ne trova un altro esistente
  # fp2tp = preds[
  #  (preds.cat == "FP") & ((preds.notes.str.contains('NV', na=False))|
  #                        (preds.notes.str.contains('NOT VISIBLE', na=False)) |
  #                        (preds.notes.str.contains('NOT VISIBILE', na=False)) |
  #                        (preds.entry_id.str.contains('neg', na=False))) &
  #                        (preds.notes.str.contains('INSIDE OTHER', na=False))].shape[0]

  # siti visibili e il modello ne trova un altro esistente
  # fp2fn = sites_inside_ot - fp2tp

  # tn_a = tn + fn2tn + fp2tp
  # fn_a = fn - fn2tn + fp2fn
  # tp_a = tp + (fp2tp + fp2fn)
  # fp_a = fp - (fp2tp + fp2fn)

  # matrix_adj=[tp_a,tn_a,fp_a,fn_a]

  # return(matrix, matrix_adj)
  return matrix

### Stampa report statistiche

In [ ]:
def stampa_stats(cm, modello, cm_adj=None):
    print('--------------------------------------------')
    print("Stats Modello " + str(modello))
    print('---')

    print("Valutazione automatica:")
    print("TP: " + str(cm[0]) + " TN: " + str(cm[1]) + " FP: " + str(cm[2]) + " FN: " + str(cm[3]))
    print("Accuracy: " + str(round((cm[0] + cm[1]) / (cm[0] + cm[1] + cm[2] + cm[3]), 4)))
    print("Recall: " + str(round(cm[0] / (cm[0] + cm[3]), 4)))

    with open(log_path, 'a') as log_file:
        log_file.write("Stats Modello " + str(modello) + "\n")
        log_file.write("Valutazione automatica: " + "TP: " + str(cm[0]) + " TN: " + str(cm[1]) + " FP: " + str(
            cm[2]) + " FN: " + str(cm[3]) + "\n")
        log_file.write("Accuracy: " + str(round((cm[0] + cm[1]) / (cm[0] + cm[1] + cm[2] + cm[3]), 4)) + "\n")
        log_file.write("Recall: " + str(round(cm[0] / (cm[0] + cm[3]), 4)) + "\n")

    if cm_adj is not None:
        print('---')

        print("Valutazione manuale:")
        print("TP: " + str(cm_adj[0]) + " TN: " + str(cm_adj[1]) + " FP: " + str(cm_adj[2]) + " FN: " + str(cm_adj[3]))
        print("Accuracy: " + str(round((cm_adj[0] + cm_adj[1]) / (cm_adj[0] + cm_adj[1] + cm_adj[2] + cm_adj[3]), 4)))
        print("Recall: " + str(round(cm_adj[0] / (cm_adj[0] + cm_adj[3]), 4)))

        with open(log_path, 'a') as log_file:
            log_file.write("Valutazione manuale: \n")
            log_file.write("TP: " + str(cm_adj[0]) + " TN: " + str(cm_adj[1]) + " FP: " + str(
                cm_adj[2]) + " FN: " + str(cm_adj[3]) + "\n")
            log_file.write("Accuracy: " + str(round((cm_adj[0] + cm_adj[1]) / (cm_adj[0] + cm_adj[1] + cm_adj[2] + cm_adj[3]), 4)) + "\n")
            log_file.write("Recall: " + str(round(cm_adj[0] / (cm_adj[0] + cm_adj[3]), 4)) + "\n")

    print('--------------------------------------------')


In [ ]:
for modello in modelli_da_comparare:
  PATH_GEOJSON_02 = base_url + '/outputs/'+ modello +'/GeoJson/preds02.geojson'
  PATH_GEOJSON_05 = base_url + '/outputs/'+ modello +'/GeoJson/preds05.geojson'

  print("\nCutoff: 0.2")
  preds = geopd.read_file(PATH_GEOJSON_02).sort_values("index").reset_index(drop=True)
  confusion_matrix = calculate_matrix(preds)
  with open(log_path, 'a') as log_file:
        log_file.write("\nCutoff: 0.2")

  stampa_stats(confusion_matrix, modello, None)






  print("\nCutoff: 0.5")
  preds = geopd.read_file(PATH_GEOJSON_05).sort_values("index").reset_index(drop=True)
  confusion_matrix = calculate_matrix(preds)
  with open(log_path, 'a') as log_file:
        log_file.write("\nCutoff: 0.5")
  stampa_stats(confusion_matrix, modello, None)



